In [1]:
import glob 
import os
import cv2
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from sklearn.metrics import confusion_matrix

Populating the interactive namespace from numpy and matplotlib


In [56]:
files = glob.glob('./resnet50_segnet_5epoch_200/*.png')
for item in files:
    img = cv2.imread(item, cv2.IMREAD_GRAYSCALE)
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=8)
    sizes = stats[1:, -1]; nb_components = nb_components - 1
    if len(sizes)!= 0:
        target_size = sizes[-1]
    img2 = np.zeros((output.shape))
    for i in range(0, nb_components):
        if sizes[i] == target_size:
            img2[output == i + 1] = 255
    cv2.imwrite('./resnet50_segnet_5epoch_200_postpro/'+item.split('/')[-1],img2)

# Calculate the mean percent of missed pixel and extras pixles  

In [3]:
from sklearn.metrics import confusion_matrix
total_miou = 0
total_dice = 0
total_fpr = 0
total_tpr = 0
# img_out = glob.glob('/home/sara/resnet50_segnet_5epoch/*.png')
img_out = glob.glob('./resnet50_segnet_5epoch_200_postpro/*.png')
img_label = glob.glob('./data1/test_label1/*.png')
total = 0
for pic in range(0,len(img_out)):
   
    m_out=cv2.imread(img_out[pic])
#     print(np.unique(m_out))
    prediction=np.reshape(np.where(m_out==255,1,0),(-1))
    target=np.reshape(cv2.imread(img_label[pic]),(-1))

    cm = confusion_matrix(target,prediction)
    tn = cm[0][0] #true positive
    fp = cm[0][1] #false positive
    fn = cm[1][0] #false negative
    tp = cm[1][1] #true positive

    total_miou = 0.5*((tn/(tn+fp+tn+fn-tn)) + (tp/(fn+tp+fp+tp-tp))) + total_miou
    total_dice = ((tn/(tn+fp+tn+fn)) + (tp/(fn+tp+fp+tp))) + total_dice
    tpr = (tp/(tp + fn)) * 100
    fpr = (fp/(fp + tn)) * 100
#     print('tpr: ',tpr,'\n')
#     print('fpr: ',fpr,'\n')
#     total_fpr = ((fp/(fp + tn)) * 100) + total_fpr
#     total_tpr = ((tp/(tp + fn)) * 100) + total_tpr
    total_fpr = fpr + total_fpr
    total_tpr = tpr + total_tpr
print('total_miou:',(total_miou/len(img_out)),'\n')
print('total_dice*',(total_dice/len(img_out)),'\n')
print('total_fpr:',(total_fpr/len(img_out)),'\n')
print('total_tpr:',(total_tpr/len(img_out)),'\n')

total_miou: 0.8424005938812917 

total_dice* 0.8854218990623124 

total_fpr: 0.2067357072882769 

total_tpr: 79.63620676486015 



In [29]:
from sklearn.metrics import confusion_matrix
total_miou = 0
img_out = glob.glob('./output/*.png')
img_label = glob.glob('./data1/test_label1/*.png')
total = 0
for pic in range(0,len(img_out)):
   
    m_out=cv2.imread(img_out[pic])
    prediction=np.reshape(np.where(m_out>0,1,0),(-1))
    target=np.reshape(cv2.imread(img_label[pic]),(-1))
#     print(type(target))
#     print(type(prediction))
    cm = confusion_matrix(target,prediction)
    tn = cm[0][0] #c00
    fp = cm[0][1] #c01
    fn = cm[1][0]
    tp = cm[1][1]
#     print(tn,fp,fn,tp)
    total_miou = 0.5*((tn/(tn+fp+tn+fn-tn)) + (tp/(fn+tp+fp+tp-tp))) + total_miou
#     print(total_miou)
print('total_mean',(total_miou/len(img_out)))

total_mean 0.7013481658310319
